In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(website_url)

In [4]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

In [5]:
table = soup.find('table',{'class': 'wikitable sortable'})

In [6]:
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [7]:
# Build parser
n_columns=0
n_rows=0
column_names= []

row_marker= 0 

# Find number of rows and columns
# also find columns header names 
for row in table.find_all('tr'):
    td_tags= row.find_all('td')
    if len(td_tags)>0:
        n_rows+=1
        if n_columns==0:
            n_columns== len(td_tags)
            
    #Handle column names
    th_tags= row.find_all('th')
    if len(th_tags)>0 and len(column_names)==0:
        for th in th_tags:
            column_names.append(th.get_text())

columns= column_names if len(column_names)>0 else range(0,n_columns)
df= pd.DataFrame(columns= columns, index= range(0, n_rows))
row_marker=0
for row in table.find_all('tr'):
    column_marker=0
    columns= row.find_all('td')
    for column in columns:
        df.iloc[row_marker, column_marker]= column.get_text()
        column_marker+=1
    if len(columns)>0:
        row_marker+=1

# Preview of Wikipedia postal codes as a panda DataFrame
df.head()
        
        
            

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [8]:
df.shape

(289, 3)

In [9]:
df= df.drop(df[df.Borough == 'Not assigned'].index).reset_index(drop=True)

In [10]:
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n
5,M6A,North York,Lawrence Manor\n
6,M7A,Queen's Park,Not assigned\n
7,M9A,Etobicoke,Islington Avenue\n
8,M1B,Scarborough,Rouge\n
9,M1B,Scarborough,Malvern\n


In [11]:
df.rename(columns={'Neighbourhood\n': 'Neighbourhood'}, inplace=True)
df.replace('\n', '', regex=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [12]:
dfgrouped= df.groupby(['Postcode', 'Borough'], as_index=False, sort=False)['Neighbourhood'].apply(','.join).reset_index(name='Neighbourhood')
dfgrouped= pd.DataFrame(dfgrouped)
dfgrouped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [13]:
dfgrouped['Neighbourhood'].loc[dfgrouped['Neighbourhood']=='Not assigned']= dfgrouped['Borough']

In [14]:
# Check if any neighbourhoods left with unassigned values
dfgrouped['Neighbourhood'].loc[dfgrouped['Neighbourhood']=='Not assigned'].count()

0

In [15]:
dfgrouped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [16]:
dfgrouped.shape

(103, 3)

# Part 2: Adding geospatial coordinates


### Let's add latitude and longitude coordinates for each neighbourhood

In [18]:
df_geo= pd.read_csv('Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_geo.shape

(103, 3)

In [20]:
df_geo= df_geo.rename(columns={'Postal Code': 'Postcode'})
df_geo.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

In [21]:
df2= pd.merge(dfgrouped,
            df_geo[['Postcode','Latitude', 'Longitude']],
             on='Postcode')
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [22]:
df2.shape

(103, 5)

In [23]:
df2.isnull().any()

Postcode         False
Borough          False
Neighbourhood    False
Latitude         False
Longitude        False
dtype: bool

# Part 3: Explore and cluster

## Select neighbourhoods from dataframe containing the word 'Toronto'

In [24]:
df_narrowed= df2[df2['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_narrowed.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [25]:
address= 'Toronto, Ontario'
geolocator= Nominatim()
location= geolocator.geocode(address)
latitude= location.latitude
longitude= location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

C:\Miniconda3\envs\COURSERA\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geographical coordinates of Toronto are 43.653963, -79.387207.


In [26]:
toronto_map= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_narrowed['Latitude'],df_narrowed['Longitude'],df_narrowed['Neighbourhood']):
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(toronto_map)

toronto_map

In [27]:
# @hidden_cell
CLIENT_ID = 'DJ4IBIT2FYCPKUX4YIPM2XQ1H1XQIV35NRSAJDI4ZL2K5BYW' # your Foursquare ID
CLIENT_SECRET = 'KIUEA452WTLUQCBK3UUGE1KWJBYXTSPXKFIFVRB3TBOHSB1G' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DJ4IBIT2FYCPKUX4YIPM2XQ1H1XQIV35NRSAJDI4ZL2K5BYW
CLIENT_SECRET:KIUEA452WTLUQCBK3UUGE1KWJBYXTSPXKFIFVRB3TBOHSB1G


In [28]:
df3= df_narrowed
df3.loc[0, 'Neighbourhood']

'Harbourfront,Regent Park'

In [29]:
neighbourhood_latitude= df3.loc[0, 'Latitude']
neighbourhood_longitude= df3.loc[0, 'Longitude']

neighbourhood_name= df3.loc[0, 'Neighbourhood']

print('The latitude and longitude of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

The latitude and longitude of Harbourfront,Regent Park are 43.6542599, -79.3606359.


In [30]:
LIMIT=100
radius=500
url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=DJ4IBIT2FYCPKUX4YIPM2XQ1H1XQIV35NRSAJDI4ZL2K5BYW&client_secret=KIUEA452WTLUQCBK3UUGE1KWJBYXTSPXKFIFVRB3TBOHSB1G&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [31]:
results= requests.get(url).json()

In [32]:
# create function to find category type
def get_category_type(row):
    try:
        categories_list= row['categories']
    except:
        categories_list= row['venue.categories']
    if len(categories_list)==0:
            return None
    else:
        return categories_list[0]['name']

In [33]:
venues= results['response']['groups'][0]['items']

nearby_venues= json_normalize(venues)

#filter columns
columns_filtered= ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

nearby_venues= nearby_venues.loc[:, columns_filtered]
#filter the category for each row
nearby_venues['venue.categories']= nearby_venues.apply(get_category_type, axis=1)

#clean up column titles
nearby_venues.columns= [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874
5,Impact Kitchen,Restaurant,43.656369,-79.356980
6,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
7,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
8,Corktown Common,Park,43.655618,-79.356211
9,The Distillery Historic District,Historic Site,43.650244,-79.359323


In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
toronto_frame= getNearbyVenues(names= df3['Neighbourhood'], latitudes= df3['Latitude'], longitudes=df3['Longitude'])

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


In [36]:
print(toronto_frame.shape)

(1707, 7)


In [37]:
toronto_frame.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [38]:
toronto_frame.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton,Exhibition Place,Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",48,48,48,48,48,48
Central Bay Street,83,83,83,83,83,83
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [39]:
print('there are {} uniques categories.'.format(len(toronto_frame['Venue Category'].unique()))) 

there are 242 uniques categories.


In [40]:
# Set pandas viewing option
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [41]:
# One hot encoding 
toronto_onehot= pd.get_dummies(toronto_frame[['Venue Category']], prefix="", prefix_sep="")
# Neighborhood exists in Dummie categories, so let's change its values to the ones in the Neighborhood dataframe
toronto_onehot['Neighborhood']= toronto_frame['Neighborhood']

#Move Neighborhood column to front of dataframe
cols=toronto_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot=toronto_onehot.reindex(cols, axis='columns')
toronto_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Harbo

In [42]:
toronto_onehot.shape

(1707, 242)

In [43]:
toronto_grouped= toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.0,0.0,0.0,0.010000,0.01,0.0,0.03,0.0,0.000000,0.000000,0.0,0.000000,0.030000,0.0,0.030000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.02,0.000000,0.01,0.020000,0.000000,0.0,0.01,0.02,0.010000,0.0,0.000000,0.050000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.030000,0.010000,0.060000,0.0,0.0,0.0,0.000000,0.000000,0.020000,0.000000,0.020000,0.0,0.0,0.000000,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.000000,0.

In [44]:
toronto_grouped.shape

(38, 242)

## Let's print the top 5 venues for each neighbourhood


In [45]:
top_venues= 5

for hood in toronto_grouped['Neighborhood']:
    print('___'+ hood+'___')
    temp= toronto_grouped[toronto_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns= ['Venue', 'Freq']
    
    temp= temp.iloc[1:]
    temp['Freq']= temp['Freq'].astype(float)
    temp= temp.round({'Freq': 2})
    print(temp.sort_values(by= 'Freq', ascending=False).reset_index(drop=True).head(top_venues))

___Adelaide,King,Richmond___
                 Venue  Freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3           Steakhouse  0.04
4      Thai Restaurant  0.04
___Berczy Park___
          Venue  Freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2    Restaurant  0.05
3   Cheese Shop  0.04
4          Café  0.04
___Brockton,Exhibition Place,Parkdale Village___
                  Venue  Freq
0        Breakfast Spot  0.11
1                  Café  0.11
2           Coffee Shop  0.11
3                   Bar  0.05
4  Caribbean Restaurant  0.05
___Business Reply Mail Processing Centre 969 Eastern___
                Venue  Freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2          Restaurant  0.06
3       Burrito Place  0.06
4             Butcher  0.06
___CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara___
              Venue  Freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airpor

### Let's put that into a pandas Dataframe

In [46]:
def most_common_category(row, num_top_venues):
    row_category= row.iloc[1:]
    row_categories_sorted= row_category.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues= 10

indicators= ['st','nd','rd']

# create columns according to top venues.
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))    
    except:
        columns.append('{}{} Most Common Venue'.format(ind+1,'th'))

# create new dataframe
top_venues_sorted= pd.DataFrame(columns= columns)
top_venues_sorted['Neighborhood']= toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    top_venues_sorted.iloc[ind, 1:]= most_common_category(toronto_grouped.iloc[ind,:], num_top_venues)
    
top_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Clothing Store,Gym,Hotel,Bakery,Bar
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Pub,Seafood Restaurant,Italian Restaurant,Beer Bar
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Coffee Shop,Café,Bar,Furniture / Home Store,Nightclub,Italian Restaurant,Climbing Gym,Burrito Place,Stadium
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden,Comic Shop,Pizza Place,Park,Recording Studio,Restaurant,Butcher,Burrito Place
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Plane,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boutique,Boat or Ferry
5,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Bakery,Market,Café,Pub,Pizza Place,Italian Restaurant,Park,Pharmacy
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Chinese Restaurant,Burger Joint,Middle Eastern Restaurant,Ice Cream Shop,Indian Restaurant,Spa
7,"Chinatown,Grange Park,Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Bakery,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Gaming Cafe
8,Christie,Café,Grocery Store,Park,Convenience Store,Restaurant,Baby Store,Nightclub,Diner,Italian Restaurant,Athletics & Sports
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Pub,Mediterranean Restaurant,Gastropub,Fast Food Restaurant


# Cluster neighborhoods

In [48]:
# set number of clusters
k_clusters= 5

toronto_grouped_clustering= toronto_grouped.drop('Neighborhood',1)

#instantiate kmeans model
kmeans_toronto= KMeans(n_clusters= k_clusters, random_state=123).fit(toronto_grouped_clustering)

#check labels
kmeans_toronto.labels_.shape

(38,)

In [49]:
toronto_merged= df3

toronto_merged['Cluster Label']= kmeans_toronto.labels_

toronto_merged= toronto_merged.join(top_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Bakery,Café,Pub,Park,Theater,Mexican Restaurant,Restaurant,Breakfast Spot,Electronics Store
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Plaza,Restaurant,Ramen Restaurant,Tea Room,Thai Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Cocktail Bar,Cosmetics Shop,Bakery,Park,Italian Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Health Food Store,Pub,Park,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Restaurant,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Pub,Seafood Restaurant,Italian Restaurant,Beer Bar


In [50]:
clusters_map= folium.Map(location=[latitude, longitude], zoom_start=11)

#set color scheme for clusters
x= np.arange(k_clusters)
ys= [(i+x+(i*x)**2)for i in range(k_clusters)]
colors_array= cm.rainbow(np.linspace(0,1, len(ys)))
rainbow= [colors.rgb2hex(i) for i in colors_array]

# place clusters on map
markers_colors=[]

for lat, long, poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'],toronto_merged['Neighbourhood'],toronto_merged['Cluster Label']):
    label= folium.Popup(str(poi)+ ' Cluster ' +str(cluster))
    folium.CircleMarker(
    [lat, long],
    radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(clusters_map)
    
clusters_map



## Examine clusters

### Cluster 1 : Mainly coffee shop area

In [51]:
toronto_merged.loc[toronto_merged['Cluster Label']==0, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Café,Pub,Park,Theater,Mexican Restaurant,Restaurant,Breakfast Spot,Electronics Store
1,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Plaza,Restaurant,Ramen Restaurant,Tea Room,Thai Restaurant
2,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Cocktail Bar,Cosmetics Shop,Bakery,Park,Italian Restaurant
3,East Toronto,0,Coffee Shop,Health Food Store,Pub,Park,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Bar,Chinese Restaurant,Burger Joint,Middle Eastern Restaurant,Ice Cream Shop,Indian Restaurant,Spa
6,Downtown Toronto,0,Café,Grocery Store,Park,Convenience Store,Restaurant,Baby Store,Nightclub,Diner,Italian Restaurant,Athletics & Sports
7,Downtown Toronto,0,Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Clothing Store,Gym,Hotel,Bakery,Bar
8,West Toronto,0,Bakery,Pharmacy,Supermarket,Gym / Fitness Center,Pizza Place,Pool,Music Venue,Discount Store,Café,Middle Eastern Restaurant
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Pizza Place,Italian Restaurant,Bakery,Scenic Lookout,Restaurant,Brewery
10,West Toronto,0,Bar,Men's Store,Asian Restaurant,Coffee Shop,Cocktail Bar,Pizza Place,Vietnamese Restaurant,Restaurant,Bakery,Café


### Cluster 2 - Mainly Playground, Outdoor Recreation area

In [52]:
toronto_merged.loc[toronto_merged['Cluster Label']==1, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,1,Playground,Tennis Court,Gym,Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store


### Cluster 3 - Bar area

In [53]:
toronto_merged.loc[toronto_merged['Cluster Label']==2, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,2,Coffee Shop,Restaurant,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Pub,Seafood Restaurant,Italian Restaurant,Beer Bar


### Cluster 4 - Italian, Oriental

In [54]:
toronto_merged.loc[toronto_merged['Cluster Label']==3, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,3,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Park,Chinese Restaurant,Rental Car Location,Dessert Shop,Salon / Barbershop,Sandwich Place
27,West Toronto,3,Pizza Place,Café,Coffee Shop,Italian Restaurant,Sushi Restaurant,Gourmet Shop,Falafel Restaurant,Park,Food,Bar


### Cluster 5 - Central Bus area

In [55]:
toronto_merged.loc[toronto_merged['Cluster Label']==4, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,4,Bus Line,Park,Swim School,Dim Sum Restaurant,Yoga Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
